In [ ]:
import pandas as pd
import numpy as np
from haversine import haversine
import os

df = pd.read_csv('4_summary.txt')
IDs = df['od_id'].values

# make directory
if not os.path.isdir('./preprocessing'):
    os.mkdir('./preprocessing')

# partition    
count = 0
num_IDs = len(IDs)
IDs_in_onefile = 10000
num_file = int(num_IDs / IDs_in_onefile) + 2

for j in range(1, num_file, 1):
#make file
    f =open("./preprocessing/preprocessing_{}.txt".format(j), 'a')
    for i in range(count, IDs_in_onefile*j, 1):

#if not exist file, skip
        if not os.path.isfile('/home/heejung/Dropbox/2019 현대MNS ETA/04. 개발/id_based_parsing/id_{}.csv'.format(IDs[count])):
            print('id_{}.csv can not be found'.format(IDs[count]))
            count = count + 1
            continue
            
#read id file            
        df1 = pd.read_csv('/home/heejung/Dropbox/2019 현대MNS ETA/04. 개발/id_based_parsing/id_{}.csv'.format(IDs[count]))
        print(count, IDs[count])
        df2 = df1.loc[0]
        
# lngs, lats
        df2['lngs'] = df1['lon'].values
        df2['lats'] = df1['lat'].values

# time, dist
        tot_time = df1.loc[df1.index[-1], 'track_sum_travel_time']
        tot_dist = df1.loc[df1.index[-1], 'sum_len']
        df2['time'] = tot_time / 10
        df2['dist'] = tot_dist / 10000

# to datetime
        time = df1.loc[0, 'track_in_time']
        date = df1.loc[0, "track_in_date"]
        test = str(date) + str('%6d' %time)
        test = test.replace(' ', '0')
        test = pd.to_datetime(test)

# weekID, dateID, timeID
        weekID = test.weekday()
        df2['weekID'] = weekID
        df2['dateID'] = test.day
        timeID = (test.hour * 60) + (test.minute) + int(test.second / 60)
        df2['timeID'] = timeID

# driverID
        df2 = df2.rename({'od_id': 'driverID'})
        driver =df2['driverID']
        driver = driver.replace('B', '')
        driver = driver[12:]
        df2['driverID'] = int(driver)

# dist gap
        distgap = (df1['sum_len'] - df1['flink_len'])/10000
        dist = []
        for i in range(0, len(df1['lon']), 1):
            if(i < len(df1['lon']) - 1):
                dist.append(haversine((df1.loc[i,'lon'], df1.loc[i, 'lat']), (df1.loc[i+1, 'lon'], df1.loc[i+1, 'lat'])))
        dist = pd.Series(dist)
        flink_index = np.array(df1.index[df1['index'] == 0])
        lis = np.array(df1.index)
        idx_del = np.delete(lis, flink_index)
        for i in idx_del:
            distgap[i] = distgap[i-1] +dist[i-1]

# time gap
        hour = (df1['point_in_time'] / 10000).astype(int)
        minute = (df1['point_in_time'] / 100).astype(int) - hour * 100
        second = df1['point_in_time'].astype(int) - minute * 100 - hour * 10000
        timegap = hour * 60 * 60 + minute * 60 + second
        timegap = timegap - timegap[0]
        timegap = timegap.astype(float)
        df2['time_gap'] = timegap.values
        df2['dist_gap'] = distgap.values
    
# states == 1.0
        states = [1.0] * len(df2['lngs'])
        df2['states'] = states

# drop columns
        df2 = df2.drop(['Unnamed: 0', 'lon', 'lat', 'index', 'point_in_time', 'seq', 'flink', 'tlink', 'turn', 'flink_len', 'sum_len',
             'facil', 'road_type', 'link_type', 'light', 'lane', 'outlink_cnt', 'zone', 'track_in_stamp', 'track_in_date',
             'track_in_time', 'track_speed', 'track_travel_time', 'track_sum_travel_time', 'static_spd', 'hq_speed',
             'ol_rot_spd', 'ol_spd', 'svc_name'])

#write file
        f.write(df2.to_json())
        f.write('\n')
        count = count + 1
    f.close()
    print('file was made')